In [1]:
import psycopg2 as pg
from psycopg2 import Error
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from pandas_profiling import ProfileReport
import json

In [2]:
def get_data_from_sql_as_df(query):
    dbname='etltestdb' 
    user='postgres' 
    host='localhost' 
    port='5432' 
    password='P@ssw0rd'
    engine = pg.connect(f"dbname= {dbname} user={user} host={host} port={port} password={password}")
    df = pd.read_sql(query, con=connection)
    return df

def get_data_from_sql(connection, query):
    df = pd.read_sql(query, con=connection)
    return df

In [3]:
with open("config.json") as f:
    source_target_data = json.load(f)

In [4]:
source_target_data

[{'source': 'sql query/SOURCE.txt',
  'target': 'sql query/TARGET.txt',
  'dbname': 'etltestdb',
  'user': 'postgres',
  'host': 'localhost',
  'port': '5432',
  'password': 'P@ssw0rd',
  'paramter1': 'BNK',
  'file_name': 'source1.csv',
  'profile_path': 'output1.html'},
 {'source': 'sql query/SOURCE.txt',
  'target': 'sql query/TARGET.txt',
  'dbname': 'etltestdb',
  'user': 'postgres',
  'host': 'localhost',
  'port': '5432',
  'password': 'P@ssw0rd',
  'paramter1': 'BNK',
  'file_name': 'source2.csv',
  'profile_path': 'output2.html'}]

In [5]:
def compare_source_and_target_data(source_path, target_path, source_connection,target_connection, parameter):
    with open(source_path) as f:
        query = f.read()
    source_df = get_data_from_sql(source_connection, query.format(parameter))
    
    with open(target_path) as f:
        query = f.read()
    target_df = get_data_from_sql(target_connection, query.format(parameter))
    
    source_df = source_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    target_df = target_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    
    col = {'addresss':'address', 'banking_type_dc': 'banking_type'}
    source_df = source_df.rename(columns  = col)

    col = {'cif':'customer_id'}
    target_df = target_df.rename(columns  = col)
    
    merge = source_df.merge(target_df, how = 'outer' ,indicator=True)
    missing_data_from_source = merge.loc[lambda x : x['_merge']=='left_only']
    missing_data_from_target = merge.loc[lambda x : x['_merge']=='right_only']
    common_data = merge.loc[lambda x : x['_merge']=='both']
    
    missing_data_from_source.drop(["_merge"], axis = 1, inplace = True)
    missing_data_from_source.reset_index(inplace = True,drop = True)

    missing_data_from_target.drop(["_merge"], axis = 1, inplace = True)
    missing_data_from_target.reset_index(inplace = True,drop = True)

    common_data.drop(["_merge"], axis = 1, inplace = True)
    common_data.reset_index(inplace = True,drop = True)
    
    missing_data_from_source = update_comments(target_df, missing_data_from_source)
    return missing_data_from_source, missing_data_from_target, common_data


In [6]:
def update_comments(actual_df, missing_df):
    common_df =actual_df[actual_df.customer_id.isin(missing_df.customer_id.values)]
    common_df.reset_index(inplace = True,drop = True)
    cols = missing_df.columns
    missing_df["Comments"] = ""
    for i in range(0,common_df.shape[0]):
        lst =[]
        for col in cols:
            if col not in common_df.columns:
                lst.append(f"'{col}'")
            else:
                if common_df.iloc[[0]][col][0] != missing_df.iloc[[0]][col][0]:
                    lst.append(f"'{col}'")
        if len(lst) > 0:
            missing_df.loc[i, "Comments"] = f"Affected columns: {','.join(lst)}"
    return missing_df

In [7]:
for data in source_target_data:
    source_path= data["source"]
    target_path= data["target"]
    dbname= data["dbname"] 
    user= data["user"] 
    host=data["host"]
    port=data["port"]
    password=data["password"]
    parameter = data["paramter1"]
    file_name = data["file_name"]
    profile_path = data["profile_path"]
    source_connection = pg.connect(f"dbname= {dbname} user={user} host={host} port={port} password={password}")
    target_connection = pg.connect(f"dbname= {dbname} user={user} host={host} port={port} password={password}")

    missing_data_from_source, missing_data_from_target, common_data = compare_source_and_target_data(source_path, target_path, source_connection,target_connection, parameter)
    missing_data_from_source.to_csv(file_name)
    
    cols = ["customer_id", "customer_name"]
    prof = ProfileReport(missing_data_from_source[cols])
    prof.to_file(output_file=profile_path)

OperationalError: could not connect to server: Connection refused (0x0000274D/10061)
	Is the server running on host "localhost" (::1) and accepting
	TCP/IP connections on port 5432?
could not connect to server: Connection refused (0x0000274D/10061)
	Is the server running on host "localhost" (127.0.0.1) and accepting
	TCP/IP connections on port 5432?


In [17]:
print(missing_data_from_source.shape)
missing_data_from_source.head()

(6, 9)


,customer_id,customer_name,address,banking_type,account_number,joint_holder,account_type,time_key,Comments
0,3584685,Aman,",DFDF,DSFF",BNK,2132433,test122,CURRENT,2020-01-01,Affected columns: 'address'
1,3584685,Aman,",DFDF,DSFF",BNK,2132433,test122,SAVINGS,2020-01-01,Affected columns: 'address'
2,3584678,Anu,"ADAF,,FDAF",BNK,2524643,test222,CURRENT,2020-01-04,Affected columns: 'address'
3,3584685,Aman,",DFDF,DSFF",BNK,2132433,test122,CORPORATE,2020-01-01,Affected columns: 'address'
4,3584678,Anu,"ADAF,,FDAF",BNK,2524643,test222,SAVINGS,2020-01-04,Affected columns: 'address'


In [12]:
print(missing_data_from_target.shape)
missing_data_from_target.head()

(15, 8)


,customer_id,customer_name,address,banking_type,account_number,joint_holder,account_type,time_key
0,3584692,Mathew,"AFAF,AFADF,GGG",BNK,1123445,test11@@test21@@test31,SAVINGS,2020-01-03
1,3584699,Abin,"ASDS,EWWSD,DFDF",BNK,1123445,test1@@test2,SAVINGS,2020-01-01
2,3584685,Aman,"DFDF,DSFF",BNK,2132433,test122,CURRENT,2020-01-01
3,3584699,Abin,"ASDS,EWWSD,DFDF",BNK,2132433,test1@@test2,CURRENT,2020-01-01
4,3584699,Abin,"ASDS,EWWSD,DFDF",BNK,1123445,test1@@test2,CORPORATE,2020-01-01


In [13]:
print(common_data.shape)
common_data.head()

(7, 8)


,customer_id,customer_name,address,banking_type,account_number,joint_holder,account_type,time_key
0,3584692,Mathew,"AFAF,AFADF,GGG",BNK,2524646,test11@@test21@@test31,ELITE,2020-01-01
1,3584671,Ananya,"AFDAF,ADFF,ER3F",BNK,1123445,test322,CORPORATE,2020-01-01
2,3584692,Mathew,"AFAF,AFADF,GGG",BNK,1123445,test11@@test21@@test31,CORPORATE,2020-01-03
3,3584671,Ananya,"AFDAF,ADFF,ER3F",BNK,1123445,test322,ELITE,2020-01-01
4,3584692,Mathew,"AFAF,AFADF,GGG",BNK,1123445,test11@@test21@@test31,ELITE,2020-01-03


In [ ]:
# ipywidgets issue fix
# conda install -c conda-forge ipywidgets
# jupyter nbextension enable --py widgetsnbextension